# 12.1 Food manufacture 1
Williams, H. Paul. Model building in mathematical programming. John Wiley & Sons, 2013.

In [ ]:
from typing import Dict

import numpy as np
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

In [ ]:
path = r"food_manufacture_1.xlsx"
dfs = pd.read_excel(path, sheet_name=None, index_col=0)

## 集合の読み込み

In [ ]:
sets = {k: list(row[~row.isnull()]) for k, row in dfs["sets"].iterrows()}
oils = sets["oils"]
time_periods = sets["time_periods"]
refine_lines = sets["refine_lines"]

## パラメータの読み込み

In [ ]:
hardness = dfs["hardness"].to_dict()["Value"]
refine_line = dfs["refine_line"].to_dict()["Value"]
prod_ub = dfs["prod_ub"].to_dict()["Value"]
buy_uc = dfs["buy_uc"].to_dict(orient="index")
sell_uc = 150
stock_uc = 5
stock_ub = 1000
stock_init = 500
stock_final = 500
hardness_ub = 6
hardness_lb = 3

## モデリング

### 変数

In [ ]:
buy_amount = LpVariable.dicts("buy_amount", (oils, time_periods), lowBound=0, upBound=None, cat="Continuous")
use_amount = LpVariable.dicts("use_amount", (oils, time_periods), lowBound=0, upBound=None, cat="Continuous")
sell_amount = LpVariable.dicts("sell_amount", time_periods, lowBound=0, upBound=None, cat="Continuous")
opening_stock_amount = LpVariable.dicts("opening_stock_amount", (oils, time_periods), lowBound=0, upBound=None, cat="Continuous")
closing_stock_amount = LpVariable.dicts("closing_stock_amount", (oils, time_periods), lowBound=0, upBound=None, cat="Continuous")

### 目的関数

In [ ]:
sales = [sell_amount[t] * sell_uc for t in time_periods]
buy_cost = [buy_amount[oil][t] * buy_uc[oil][t] for t in time_periods for oil in oils]
stock_cost = [closing_stock_amount[oil][t] * stock_uc for t in time_periods for oil in oils]

In [ ]:
{k: v.value() for k, v in sell_amount.items()}

In [ ]:
total_sales = lpSum(sales)
total_buy_cost = lpSum(buy_cost)
total_stock_cost = lpSum(stock_cost)
total_cost = total_buy_cost + total_stock_cost
total_profit = total_sales - total_cost

In [ ]:
model = LpProblem("Food manufacture 1", LpMaximize)
model.setObjective(total_profit)

### 制約条件

In [ ]:
# 期初在庫、期末在庫
t_init = time_periods[0]
t_final = time_periods[-1]
for oil in oils:
    model += opening_stock_amount[oil][t_init] == stock_init
    model += closing_stock_amount[oil][t_final] == stock_final

In [ ]:
# 倉庫バランス
for i, t in enumerate(time_periods):
    for oil in oils:
        if i > 0:
            t_prev = time_periods[i - 1]
            model += opening_stock_amount[oil][t] == closing_stock_amount[oil][t_prev]
        model += closing_stock_amount[oil][t] == opening_stock_amount[oil][t] + buy_amount[oil][t] - use_amount[oil][t]

In [ ]:
# 倉庫上限
for t in time_periods:
    for oil in oils:
        model += closing_stock_amount[oil][t] <= stock_ub

In [ ]:
# 販売量バランス
for t in time_periods:
    model += sell_amount[t] == lpSum(use_amount[oil][t] for oil in oils)

In [ ]:
# 生産ライン別製錬量上限
for line in refine_lines:
    used_oils = [oil for oil in oils if refine_line[oil] == line]
    for t in time_periods:
        total_prod_amount = lpSum(use_amount[oil][t] for oil in used_oils)
        model += total_prod_amount <= prod_ub[line]

In [ ]:
# 硬度、生産ライン別上限
for t in time_periods:
    total_hardness = lpSum(hardness[oil] * use_amount[oil][t] for oil in oils)
    model += total_hardness <= hardness_ub * sell_amount[t]
    model += total_hardness >= hardness_lb * sell_amount[t]

In [ ]:
model.solve()

In [ ]:
model.objective.value()

In [ ]:
result = {oil: {t: use_amount[oil][t].value() for t in time_periods} for oil in oils}
pd.DataFrame(result)

In [ ]:
result = {oil: {t: buy_amount[oil][t].value() for t in time_periods} for oil in oils}
pd.DataFrame(result)

In [ ]:
result = {oil: {t: opening_stock_amount[oil][t].value() for t in time_periods} for oil in oils}
pd.DataFrame(result)

In [ ]:
result = {oil: {t: closing_stock_amount[oil][t].value() for t in time_periods} for oil in oils}
pd.DataFrame(result)